In [5]:
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm.notebook import tqdm
import numpy as np

In [6]:
#Load FinBERT ProsusAI/finbert, trained on financial text.

model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [4]:
def get_embedding(text):
    # Tokenize and truncate to FinBERT’s max length
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512
    )

    with torch.no_grad():
        outputs = model(**inputs)
        # Mean pooling across tokens
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

In [2]:
embeddings = []

for text in tqdm(df_texts["clean_text"], desc="Generating FinBERT embeddings"):
    try:
        emb = get_embedding(text)
        embeddings.append(emb)
    except Exception as e:
        print("Error:", e)
        embeddings.append(np.zeros(model.config.hidden_size)) 

NameError: name 'tqdm' is not defined